This is the EfficientNetB4 network for breast cancer detection using transfer learning and Adabound as an optimizer. If you want to use our codes, please cite our paper (The paper is in submission process right now) and read our license agreement file.

At first, we supposed to import all the libraries:

In [ ]:
import tensorflow as tf
import shutil
import numpy as np
import pandas as pd
import cv2
import os
import time
import matplotlib.pyplot as plt
import glob
import keras
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Flatten, ReLU
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam, SGD, Adadelta, adagrad
import efficientnet.keras as enet
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix
from keras_radam import RAdam
from keras_adabound import AdaBound

To have reproducable results we can use the followin codes:

In [ ]:
''' Seed for reproducible results '''
from keras import backend as K

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Reading the images and labels from a directory

In [ ]:
''' Read images and labels '''
print('Reading images ...')

jpgFilenamesList = glob.glob('dataset_comparison/*.jpg')

sample_img = cv2.imread(jpgFilenamesList[0], cv2.IMREAD_GRAYSCALE)
sample_shape = sample_img.shape
dataset = np.zeros((len(jpgFilenamesList), sample_shape[0], sample_shape[1]), dtype=np.float32)
labels = np.zeros(len(jpgFilenamesList), dtype=np.float32)
for i in range(len(jpgFilenamesList)):
    dataset[i] = cv2.imread(jpgFilenamesList[i], cv2.IMREAD_GRAYSCALE)
    name = jpgFilenamesList[i].split('\\')[1]
    under_split = name.split('_')[1]
    final_split = under_split.split('.')[0]
    labels[i] = np.float32(final_split)

print(dataset.shape)
print(labels.shape)
print(type(dataset))
print(type(labels))
print(dataset[0])
print(labels)
unique, counts = np.unique(labels, return_counts=True)
print('Hereeeee')
print(unique)
print(counts)
plt.imshow(dataset[0])
plt.show()

Shuffle dataset

In [ ]:
''' Shuffle dataset '''

from sklearn.utils import shuffle
dataset, labels = shuffle(dataset, labels, random_state=42)

Normalizing dataset

In [ ]:
''' Normalize images '''
print('Normalization')
dataset = dataset.astype('float32')
print(np.max(dataset))
print(np.min(dataset))
dataset /= np.float(255)
print(np.max(dataset))
print(np.min(dataset))

Make sure that the number of channels in the image are 3.

In [ ]:
''' Image channels compatible with the network '''
# 3 Channel image:
dataset = np.stack((dataset,) * 3, axis=-1)

Split train and test dataset

In [ ]:
''' Split dataset '''
x_train_1, x_test_1, y_train1, y_test1 = train_test_split(dataset, labels, test_size=0.1)


Make the labels to categorical format

In [ ]:
''' Categorical labels '''
from keras.utils import to_categorical

y_train1 = to_categorical(y_train1, num_classes=len(np.unique(labels)))
y_test1 = to_categorical(y_test1, num_classes=len(np.unique(labels)))

Build your model

In [ ]:
''' Initialization '''
input_shape = x_train_1.shape[1:]
print(input_shape)

# loading B0 pre-trained on ImageNet without final aka fiature extractor
model = enet.EfficientNetB4(include_top=False,
                            pooling='avg',
                            input_shape=(120, 120, 3),
                            # weights='imagenet'
                            )

#  building 2 fully connected layer
x = model.output

# output layer
predictions = Dense(2, activation="softmax")(x)

model_final = Model(inputs=model.input, outputs=predictions)

model_final.summary()

Depict model by the following code

In [ ]:
''' Depict model '''
model_final.summary()
from keras.utils import plot_model
import matplotlib.pyplot as plt
import os
import cv2

os.environ["PATH"] += os.pathsep + 'Your path'
plot_model(model_final, to_file='model_single.png', show_shapes=True, show_layer_names=False)
image = cv2.imread('model_single.png')
plt.imshow(image)
plt.show()

To depict the results in each epoch, you can use following function:

In [ ]:
''' Plot online results (Live results)'''
from IPython.display import clear_output


class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.acces = []
        self.val_acces = []

        self.fig = plt.figure()

        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acces.append(logs.get('acc'))
        self.val_acces.append(logs.get('val_acc'))
        self.i += 1

        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.grid(True)
        plt.show()

plot_losses = PlotLosses()

Train the model with your own model metrics, losses, and optimizers.

In [ ]:
''' Train model '''
start_time = time.time()

optm = AdaBoundOptimizer(amsbound=True,
                         learning_rate=0.001,
                         final_lr=0.1,
                         beta1=0.9,
                         beta2=0.999,
                         gamma=1e-3,
                         epsilon=1e-8,
                         use_locking=False)
# You can use your model metrics, losses, and optimizers.
# optm = Adam()
# optm = Adadelta()
# optm = adagrad()
# optm = SGD()

run_opts = tf.RunOptions(report_tensor_allocations_upon_oom=True)
model_final.compile(loss='binary_crossentropy',
                    optimizer=optm,
                    metrics=['accuracy', tf.keras.metrics.AUC(), 'binary_accuracy'])

mcp_save = ModelCheckpoint('EnetB0_CIFAR10_TL.h5', save_best_only=True, monitor='val_binary_accuracy')
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=2, verbose=1, )

print("Training....")
history = model_final.fit(x_train_1, y_train1,
                          batch_size=36,
                          epochs=30,
                          validation_split=0.1,
                          callbacks=[plot_losses, mcp_save],
                          verbose=1)

Plot and print the results:

In [ ]:
''' Best model weight results '''
print('\n\n ****** Best model weight results: ****** \n\n\n')
model_final.load_weights('EnetB0_CIFAR10_TL.h5')

y_pred = model_final.predict(x_test_1)
y_pred = np.argmax(y_pred, axis=1)
y_test12 = np.argmax(y_test1, axis=1)
cm = confusion_matrix(y_test12, y_pred)
print('Confusion Matrix')
print(cm)
tn, fp, fn, tp = confusion_matrix(y_test12, y_pred).ravel()

precision = tp / (tp + fp)
print("Precision: {}\n".format(precision))
recall = tp / (tp + fn)
print("Recall: {}\n".format(recall))

print('\nTN: {} \n FP: {} \n FN: {} \n TP: {} \n'.format(tn, fp, fn, tp))

print('Accuracy')
print(history.history['val_accuracy'])
print(np.max(history.history['val_accuracy']))

print('AUC')
# print(history.history['val_auc'])
print(np.max(history.history['val_auc']))

f1_score = 2 * (precision * recall) / (precision + recall)
print('f1_score: {}'.format(f1_score))

print("--- %s seconds ---" % (time.time() - start_time))